In [40]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [41]:
# Features: Google Review, Price, Roamify Rating
X = [[3.7, 100, 3],
    [4.5, 200, 4],
    [4.2, 150, 4.2],
    [4.3, 180, 4.1],
    [4.7, 250, 4.5],
    [4.6, 220, 4.3],
    [4.1, 120, 4],
    [4.4, 190, 4.2],
    [4.8, 280, 4.7],
    [3.9, 80, 3.8]]

num_attractions = 10
num_users = 5
num_features = 3
Y = np.random.randint(1, 6, size=(num_attractions, num_users))
W = np.random.rand(num_users, num_features)
B = np.random.rand(1,num_users)
X = np.array(X)
print("Initial Weights: ", W)
print("Initial Ratings: ", Y)
print("Shape of X: ", np.shape(X))
print("Shape of W: ", np.shape(W))
print("Shape of B: ", np.shape(B))
print("Shape of Y: ", np.shape(Y))

Initial Weights:  [[0.2945276  0.83136219 0.10846335]
 [0.37903723 0.18566696 0.91769698]
 [0.4339009  0.8753187  0.36540844]
 [0.45776368 0.85465294 0.50207773]
 [0.67864069 0.25100327 0.27942242]]
Initial Ratings:  [[1 5 2 5 1]
 [2 2 2 2 4]
 [3 3 1 4 5]
 [3 4 2 4 3]
 [2 2 4 4 5]
 [4 1 4 3 4]
 [4 1 4 4 2]
 [1 1 1 4 5]
 [1 1 3 3 5]
 [3 5 3 3 4]]
Shape of X:  (10, 3)
Shape of W:  (5, 3)
Shape of B:  (1, 5)
Shape of Y:  (10, 5)


In [42]:
def cofi_cost_func(X, W, b, Y, lambda_):
    nm, n = X.shape
    nm, nu = Y.shape
    J = 0
    for i in range(nm):
        for j in range(nu):
            # if (R[i][j] == 1):
            sum_ = 0
            for k in range(n):
                sum_ += (W[j][k] * X[i][k])
            cost = (sum_ + b[0][j] - Y[i][j]) ** 2
            J+= (cost/2)
        
    for j in range(nu):
        for k in range(n):
            cost = (W[j][k]) ** 2
            J += (cost * (lambda_ / 2))
            
    for j in range(nm):
        for k in range(n):
            cost = (X[j][k]) ** 2
            J += (cost * (lambda_ / 2))
    
        

    return J

In [43]:
print(f"Cost without regularization: {cofi_cost_func(X, W, B, Y, 0)}")
print(f"Cost with regularization: {cofi_cost_func(X, W, B, Y, 1.5)}")

Cost without regularization: 407448.14778023266
Cost with regularization: 671043.7415201532


In [44]:
tf.random.set_seed(1234)
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_attractions, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1, num_users),   dtype=tf.float64),  name='B')

optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [45]:
iterations = 200
lambda_ = 1
for iter in range(iterations):

    with tf.GradientTape() as tape:
        cost_value = cofi_cost_func(X, W, b, Y, lambda_)

    grads = tape.gradient( cost_value, [X,W,b] )

    optimizer.apply_gradients(zip(grads, [X,W,b]) )

    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 383.5
Training loss at iteration 20: 52.4
Training loss at iteration 40: 30.5
Training loss at iteration 60: 24.3
Training loss at iteration 80: 22.2
Training loss at iteration 100: 20.6
Training loss at iteration 120: 19.2
Training loss at iteration 140: 18.1
Training loss at iteration 160: 17.4
Training loss at iteration 180: 17.1


In [46]:
predicted_ratings = np.dot(X, np.transpose(W)) + b

for user in range(num_users):
    print(f"User {user+1}:")
    for attraction in range(num_attractions):
        print(f"Attraction {attraction+1}: Predicted Rating = {predicted_ratings[attraction][user]:.2f}, Actual Rating = {Y[attraction][user]}")
    print()

User 1:
Attraction 1: Predicted Rating = 1.58, Actual Rating = 1
Attraction 2: Predicted Rating = 2.20, Actual Rating = 2
Attraction 3: Predicted Rating = 2.15, Actual Rating = 3
Attraction 4: Predicted Rating = 2.56, Actual Rating = 3
Attraction 5: Predicted Rating = 2.55, Actual Rating = 2
Attraction 6: Predicted Rating = 3.53, Actual Rating = 4
Attraction 7: Predicted Rating = 3.38, Actual Rating = 4
Attraction 8: Predicted Rating = 1.04, Actual Rating = 1
Attraction 9: Predicted Rating = 1.82, Actual Rating = 1
Attraction 10: Predicted Rating = 3.11, Actual Rating = 3

User 2:
Attraction 1: Predicted Rating = 4.58, Actual Rating = 5
Attraction 2: Predicted Rating = 2.16, Actual Rating = 2
Attraction 3: Predicted Rating = 3.09, Actual Rating = 3
Attraction 4: Predicted Rating = 3.83, Actual Rating = 4
Attraction 5: Predicted Rating = 1.77, Actual Rating = 2
Attraction 6: Predicted Rating = 1.30, Actual Rating = 1
Attraction 7: Predicted Rating = 1.49, Actual Rating = 1
Attraction 8:

In [47]:
#Testing ratings
user_ratings = np.zeros(num_attractions)
user_ratings[0] = 5
user_ratings[1] = 4
user_ratings[4] = 3
user_ratings[8] = 2
user_ratings[9] = 4

user_rated = [i for i in range(num_attractions) if user_ratings[i] > 0]

Y = np.c_[user_ratings, Y]
print(Y.shape)

(10, 6)


In [48]:
num_attractions, num_users = Y.shape
num_features = 3

tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_attractions, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='B')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [49]:
iterations = 200
lambda_ = 1
for iter in range(iterations):

    with tf.GradientTape() as tape:
        cost_value = cofi_cost_func(X, W, b, Y, lambda_)

    grads = tape.gradient( cost_value, [X,W,b] )

    optimizer.apply_gradients(zip(grads, [X,W,b]) )

    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 462.1
Training loss at iteration 20: 70.5
Training loss at iteration 40: 42.4
Training loss at iteration 60: 34.0
Training loss at iteration 80: 30.8
Training loss at iteration 100: 28.2
Training loss at iteration 120: 25.9
Training loss at iteration 140: 24.2
Training loss at iteration 160: 23.3
Training loss at iteration 180: 22.9


In [50]:
p = np.dot(X,np.transpose(W)) + b

pm = p.numpy()
my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(user_ratings)):
    if user_ratings[i] > 0:
        print(f'Original {user_ratings[i]}, Predicted {my_predictions[i]:0.2f}')



Original vs Predicted ratings:

Original 5.0, Predicted 4.59
Original 4.0, Predicted 3.47
Original 3.0, Predicted 2.77
Original 2.0, Predicted 2.08
Original 4.0, Predicted 3.73
